# NLP with oil and renewable energies project

#### Get libraries

In [3]:
import requests
import re
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
import numpy as np
from PIL import Image
from sklearn.linear_model import LogisticRegression
import argparse
from matplotlib import pyplot as plt
import wordcloud
import requests, json
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import spacy
import spacy
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

### Creating dfs

In [4]:
corpus_oil_price=pd.read_csv('data/corpus_oil_price.csv', index_col=0)
corpus_alternative_energies=pd.read_csv('data/corpus_alternative_energies.csv', index_col=0)
corpus_crude_oil=pd.read_csv('data/corpus_crude_oil.csv', index_col=0)
corpus_oil_price

,title,date_published,content,baseline_sentiment,vader_sentiment
0,$70 Oil Won't Keep OPEC Happy,2021-11-29T19:00:00-06:00,One day&mdash;that was all it took oil prices ...,-1,0.9448
1,Why This Oil Price Slump May Not Be Bad News F...,2021-12-02T19:00:00-06:00,Crude oil prices are plunging under the weight...,-1,0.9957
2,The Double Irony Of Biden's Plea To OPEC,2021-11-10T13:00:00-06:00,As I wrote in February of this year in&nbsp;Th...,-1,0.8805
3,The Oil Price Rally Is Far From Over,2021-11-09T19:00:00-06:00,How much higher could oil prices go This is th...,1,0.1027
4,"Biden Threatens OPEC With Undisclosed ""Tools""",2021-11-08T10:00:00-06:00,"""There are other tools in the arsenal that we ...",-1,-0.6269
...,...,...,...,...,...
2183,A Disobliging Update on the Price of Oil,2010-01-09T16:21:21-06:00,There still seems to be a deplorable uncertain...,-1,0.9947
2184,What Happens When the Wells Run Dry,2010-01-08T16:55:39-06:00,One nagging question that the industrial world...,1,-0.7541
2185,Petro Unfriendly California Changing it's Stan...,2009-12-30T14:18:05-06:00,The financial crisis changed the way people th...,-1,0.0982
2186,New Techniques Oil Companies are Using in Dril...,2009-11-25T16:35:34-06:00,As the politics and philosophical arguments ab...,1,0.9985


In [5]:
corpus_oil_price['content'] = corpus_oil_price['content'].str.replace('"','')
corpus_oil_price['content'] = corpus_oil_price['content'].str.replace('&','')
corpus_oil_price['content'] = corpus_oil_price['content'].str.replace(';','')
corpus_oil_price

,title,date_published,content,baseline_sentiment,vader_sentiment
0,$70 Oil Won't Keep OPEC Happy,2021-11-29T19:00:00-06:00,One daymdashthat was all it took oil prices to...,-1,0.9448
1,Why This Oil Price Slump May Not Be Bad News F...,2021-12-02T19:00:00-06:00,Crude oil prices are plunging under the weight...,-1,0.9957
2,The Double Irony Of Biden's Plea To OPEC,2021-11-10T13:00:00-06:00,As I wrote in February of this year innbspThe ...,-1,0.8805
3,The Oil Price Rally Is Far From Over,2021-11-09T19:00:00-06:00,How much higher could oil prices go This is th...,1,0.1027
4,"Biden Threatens OPEC With Undisclosed ""Tools""",2021-11-08T10:00:00-06:00,There are other tools in the arsenal that we h...,-1,-0.6269
...,...,...,...,...,...
2183,A Disobliging Update on the Price of Oil,2010-01-09T16:21:21-06:00,There still seems to be a deplorable uncertain...,-1,0.9947
2184,What Happens When the Wells Run Dry,2010-01-08T16:55:39-06:00,One nagging question that the industrial world...,1,-0.7541
2185,Petro Unfriendly California Changing it's Stan...,2009-12-30T14:18:05-06:00,The financial crisis changed the way people th...,-1,0.0982
2186,New Techniques Oil Companies are Using in Dril...,2009-11-25T16:35:34-06:00,As the politics and philosophical arguments ab...,1,0.9985


In [6]:
col_one_list = corpus_oil_price['content'].tolist()

In [ ]:
common_words_list = []
for a in col_one_list:
    complete_doc = nlp(a)
    # Remove stop words and punctuation symbols
    words = [token.text for token in complete_doc
         if not token.is_stop and not token.is_punct]
    word_freq = Counter(words)
    # 5 commonly occurring words with their frequencies
    common_words = word_freq.most_common(11)
    # Unique words
    common_words_list.append(common_words)
common_words_list

In [87]:
common_words_list_edited = []
for c in common_words_list:
    intermediary_list = []
    for a in c:
        a_edited = a[0]
        intermediary_list.append(a_edited)
    common_words_list_edited.append(intermediary_list)
common_words_list_edited

[['oil',
  ' ',
  'OPEC',
  '$',
  'release',
  'new',
  'bpd',
  'million',
  'barrels',
  'rdquo',
  'prices'],
 [' ',
  'production',
  'shale',
  'oil',
  'prices',
  'approach',
  'industry',
  'drillers',
  'way',
  'plunging',
  'latest'],
 ['oil',
  ' ',
  'Biden',
  'prices',
  'Administration',
  'production',
  'mdash',
  'U.S.',
  '   ',
  'year',
  'fossil'],
 ['oil',
  ' ',
  'prices',
  'demand',
  '$',
  'higher',
  'barrel',
  'year',
  'said',
  'supply',
  'bullish'],
 [' ',
  'OPEC',
  'oil',
  'pump',
  'tools',
  'President',
  'Biden',
  'said',
  'crude',
  'Washington',
  'global'],
 ['OPEC',
  'Morse',
  'production',
  ' ',
  'oil',
  'prices',
  'said',
  'U.S.',
  'capacity',
  'year',
  'continue'],
 [' ',
  'oil',
  'million',
  'prices',
  'U.S.',
  'Fed',
  '$',
  'inflation',
  'supply',
  'crude',
  'barrels'],
 [' ',
  'crude',
  'oil',
  'barrels',
  'prices',
  'market',
  'Fed',
  'inventory',
  'API',
  'million',
  'week'],
 [' ',
  'prices',
  

In [77]:
corpus_alternative_energies['content'] = corpus_alternative_energies['content'].str.replace('"','')
corpus_alternative_energies['content'] = corpus_alternative_energies['content'].str.replace('&','')
corpus_alternative_energies['content'] = corpus_alternative_energies['content'].str.replace(';','')
col_one_list_2 = corpus_alternative_energies['content'].tolist()

In [78]:
common_words_list_2 = []
for a in col_one_list_2:
    complete_doc = nlp(a)
    # Remove stop words and punctuation symbols
    words = [token.text for token in complete_doc
         if not token.is_stop and not token.is_punct]
    word_freq = Counter(words)
    # 5 commonly occurring words with their frequencies
    common_words = word_freq.most_common(11)
    # Unique words
    common_words_list_2.append(common_words)

In [90]:
common_words_list_edited_2 = []
for c in common_words_list_2:
    intermediary_list = []
    for a in c:
        a_edited = a[0]
        intermediary_list.append(a_edited)
    common_words_list_edited_2.append(intermediary_list)
common_words_list_edited_2

[[' ',
  'energy',
  'tons',
  'metals',
  '$',
  'costs',
  'battery',
  'prices',
  'renewable',
  'copper',
  'big'],
 [' ',
  'report',
  'Southeast',
  'Asiarsquos',
  'carbon',
  'green',
  'climate',
  'Hardcastle',
  'action',
  '$',
  'Net'],
 ['energy',
  'Europe',
  ' ',
  'prices',
  'gas',
  'European',
  'transition',
  'high',
  'said',
  'EU',
  'plans'],
 ['energy',
  ' ',
  'global',
  'like',
  'transition',
  'renewable',
  'clean',
  'fossil',
  'fuels',
  'world',
  'oil'],
 [' ',
  'carbon',
  'solar',
  'oil',
  'Chevron',
  'wind',
  'energy',
  'power',
  'gas',
  'renewable',
  'emissions'],
 ['solar',
  ' ',
  'capacity',
  'energy',
  'installations',
  'GW',
  'study',
  'Energy',
  'cost',
  'U.S.',
  'government'],
 ['Manchin',
  ' ',
  'Sen.',
  'Democratic',
  '$',
  '3.5',
  'trillion',
  'plan',
  'said',
  'CNN',
  'bill'],
 ['oil',
  'major',
  ' ',
  'energy',
  'green',
  'climate',
  'hydrocarbon',
  'projects',
  'UAE',
  'power',
  'world'],
 

In [81]:
corpus_crude_oil['content'] = corpus_crude_oil['content'].str.replace('"','')
corpus_crude_oil['content'] = corpus_crude_oil['content'].str.replace('&','')
corpus_crude_oil['content'] = corpus_crude_oil['content'].str.replace(';','')
col_one_list_3 = corpus_crude_oil['content'].tolist()

In [82]:
common_words_list_3 = []
for a in col_one_list_3:
    complete_doc = nlp(a)
    # Remove stop words and punctuation symbols
    words = [token.text for token in complete_doc
         if not token.is_stop and not token.is_punct]
    word_freq = Counter(words)
    # 5 commonly occurring words with their frequencies
    common_words = word_freq.most_common(11)
    # Unique words
    common_words_list_3.append(common_words)

In [91]:
common_words_list_edited_3 = []
for c in common_words_list_3:
    intermediary_list = []
    for a in c:
        a_edited = a[0]
        intermediary_list.append(a_edited)
    common_words_list_edited_3.append(intermediary_list)
common_words_list_edited_3

[['oil',
  'prices',
  'OPEC',
  'Bidenrsquos',
  'release',
  'crude',
  'markets',
  'producers',
  'plan',
  'SPR',
  'market'],
 [' ',
  'OPEC',
  'bpd',
  'January',
  '400,000',
  'meeting',
  'production',
  'expected',
  'group',
  'cuts',
  'policy'],
 ['oil',
  'UAErsquos',
  'billion',
  'UAE',
  'US$',
  'key',
  'million',
  'deal',
  'Abu',
  'Dhabi',
  'ADNOC'],
 ['oil',
  ' ',
  'release',
  'crude',
  'reserves',
  'China',
  'prices',
  'said',
  'Chinese',
  'releasing',
  'coordinated'],
 ['million',
  ' ',
  'barrels',
  'week',
  'bpd',
  'crude',
  'draw',
  '$',
  'oil',
  'inventories',
  'previous'],
 ['release',
  'oil',
  ' ',
  'reserve',
  'barrels',
  'OPEC',
  'crude',
  'according',
  'million',
  'prices',
  'strategic'],
 ['oil',
  'ban',
  'crude',
  'exports',
  ' ',
  'House',
  'U.S.',
  'prices',
  'White',
  'Khanna',
  'potential'],
 [' ',
  'oil',
  'release',
  'million',
  'barrels',
  'SPR',
  'coordinated',
  'said',
  'OPEC',
  'Tuesday',

In [89]:
corpus_oil_price['most_common_words'] = common_words_list_edited
corpus_oil_price

,title,date_published,content,baseline_sentiment,vader_sentiment,most_common_words
0,$70 Oil Won't Keep OPEC Happy,2021-11-29T19:00:00-06:00,One daymdashthat was all it took oil prices to...,-1,0.9448,"[oil, , OPEC, $, release, new, bpd, million, ..."
1,Why This Oil Price Slump May Not Be Bad News F...,2021-12-02T19:00:00-06:00,Crude oil prices are plunging under the weight...,-1,0.9957,"[ , production, shale, oil, prices, approach, ..."
2,The Double Irony Of Biden's Plea To OPEC,2021-11-10T13:00:00-06:00,As I wrote in February of this year innbspThe ...,-1,0.8805,"[oil, , Biden, prices, Administration, produc..."
3,The Oil Price Rally Is Far From Over,2021-11-09T19:00:00-06:00,How much higher could oil prices go This is th...,1,0.1027,"[oil, , prices, demand, $, higher, barrel, ye..."
4,"Biden Threatens OPEC With Undisclosed ""Tools""",2021-11-08T10:00:00-06:00,There are other tools in the arsenal that we h...,-1,-0.6269,"[ , OPEC, oil, pump, tools, President, Biden, ..."
...,...,...,...,...,...,...
2183,A Disobliging Update on the Price of Oil,2010-01-09T16:21:21-06:00,There still seems to be a deplorable uncertain...,-1,0.9947,"[oil, price, ndash, Saudi, Watling, $, Arabia,..."
2184,What Happens When the Wells Run Dry,2010-01-08T16:55:39-06:00,One nagging question that the industrial world...,1,-0.7541,"[oil, run, begin, producing, day, thousands, p..."
2185,Petro Unfriendly California Changing it's Stan...,2009-12-30T14:18:05-06:00,The financial crisis changed the way people th...,-1,0.0982,"[oil, state, states, drilling, financial, deve..."
2186,New Techniques Oil Companies are Using in Dril...,2009-11-25T16:35:34-06:00,As the politics and philosophical arguments ab...,1,0.9985,"[drilling, oil, wells, new, rdquo, technology,..."


In [92]:
corpus_alternative_energies['most_common_words'] = common_words_list_edited_2
corpus_crude_oil['most_common_words'] = common_words_list_edited_3

In [93]:
corpus_oil_price.to_csv('data/corpus_oil_price_3.csv')
corpus_alternative_energies.to_csv('data/corpus_alternative_energies_3.csv')
corpus_crude_oil.to_csv('data/corpus_crude_oil_3.csv')